In [22]:
include("../lib/Lib.jl")
include("../lib/DeepSpace.jl")

Main.DeepSpace

In [23]:
using DataFrames, DataFramesMeta
using Statistics
using ScikitLearn
using Main

@sk_import linear_model: LinearRegression
@sk_import linear_model: ElasticNet

PyObject <class 'sklearn.linear_model.coordinate_descent.ElasticNet'>

In [24]:
KEY = "2019week0"
YEAR = "2019"

"2019"

In [25]:
(headers, scores, teams) = Lib.read_stacked(KEY, YEAR)

(["adjustPoints", "autoPoints", "bay1", "bay2", "bay3", "bay4", "bay5", "bay6", "bay7", "bay8"  …  "techFoulCount", "teleopPoints", "topLeftRocketFar", "topLeftRocketNear", "topRightRocketFar", "topRightRocketNear", "totalPoints", "key", "level", "event"], 58×54 DataFrame. Omitted printing of 50 columns
│ Row │ adjustPoints │ autoPoints │ bay1          │ bay2          │
│     │ Int64⍰       │ Int64⍰     │ String⍰       │ String⍰       │
├─────┼──────────────┼────────────┼───────────────┼───────────────┤
│ 1   │ 0            │ 9          │ Panel         │ Panel         │
│ 2   │ 0            │ 12         │ Panel         │ PanelAndCargo │
│ 3   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 4   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 5   │ 1            │ 0          │ Panel         │ Panel         │
│ 6   │ 0            │ 0          │ Panel         │ Panel         │
│ 7   │ 1            │ 0          │ Panel         │ Panel         │
│ 8   │ 0      

In [26]:
DeepSpace.calculated_columns!(scores)

58-element Array{Int64,1}:
  4
  0
  0
  0
  0
  0
  0
  0
  0
  9
  0
  0
  0
  ⋮
  0
  4
  2
  0
  0
  0
  0
  0
  0
 10
  0
 12

In [27]:
scores_train = @linq scores |> where(:level .== "qm")
teams_train = @linq teams |> where(:level .== "qm")

scores_test = @linq scores |> where(:level .!= "qm")
teams_test = @linq teams |> where(:level .!= "qm")

(ß, teamnums, teamindex) = Lib.opr_ß(scores_train, teams_train ; shortevent = true)

([1.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Union{Missing, String}["frc1073", "frc1153", "frc1517", "frc1721", "frc1729", "frc190", "frc2084", "frc2342", "frc246", "frc3236"  …  "frc3986", "frc4905", "frc501", "frc509", "frc5687", "frc58", "frc5813", "frc6153", "frc7314", "frc811"], Dict("frc1721"=>4,"frc6153"=>20,"frc3236"=>10,"frc509"=>16,"frc3467"=>12,"frc5687"=>17,"frc1153"=>2,"frc1073"=>1,"frc1729"=>5,"frc2084"=>7…))

In [28]:
model = LinearRegression(fit_intercept=false)
components = DataFrame([teamnums], [:team])

for col in Lib.numeric_col_names(scores_train)
    y = convert.(Float64, coalesce(scores_train[col], -1.0))
    modelfit = ScikitLearn.fit!(model, ß, y)
    components[col] = modelfit[:coef_]
end
components

,team,adjustPoints,autoPoints,cargoPoints,completeRocketRankingPoint,completedRocketFar,completedRocketNear,foulCount,foulPoints,habClimbPoints,habDockingRankingPoint,hatchPanelPoints,rp,sandStormBonusPoints,techFoulCount,teleopPoints,totalPoints,rocketLowPoints,rocketMidPoints,rocketTopPoints,rocketPoints
,String⍰,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,frc1073,0.0,1.14393,-3.89734,0.0,0.0,0.0,-0.0727926,-1.5973,-2.41895,-0.217654,-1.80075,0.807215,1.14393,-0.263667,-8.11704,-8.57041,0.398869,-0.602105,-0.538165,-0.741401
2,frc1153,0.0,1.48444,8.05003,0.0,0.0,0.0,-0.0737292,-1.27691,3.98808,0.11703,3.08258,1.03396,1.48444,0.499158,15.1207,15.3282,1.24248,0.0830439,-0.756906,0.568622
3,frc1517,0.0,3.87995,6.62942,0.0,0.0,0.0,0.0506349,-0.149519,0.646393,-0.0807191,1.2274,0.957621,3.87995,0.100965,8.50321,12.2336,0.880955,0.0593492,-0.126754,0.81355
4,frc1721,0.0,3.23734,-11.1395,0.0,0.0,0.0,0.110384,3.18017,0.913833,0.0194458,2.83401,-0.206687,3.23734,-0.251238,-7.39169,-0.974181,1.18423,1.40153,0.322903,2.90867
5,frc1729,0.0,5.10924,4.05574,0.0,0.0,0.0,-0.0372124,2.30582,8.50868,0.0576267,-0.527798,-0.404448,5.10924,0.447494,12.0366,19.4517,1.54817,-0.910057,0.0416279,0.679737
6,frc190,0.0,1.07835,-4.72199,0.0,0.0,0.0,-0.265043,-1.61334,1.34039,-0.24522,-0.163504,-0.784452,1.07835,-0.10842,-3.5451,-4.08008,-0.8875,-0.496676,0.5715,-0.812677
7,frc2084,0.0,3.7338,9.66709,0.0,0.0,0.0,0.377197,-3.60934,0.108556,-0.0971937,-2.16665,0.611937,3.7338,0.530647,7.609,7.73346,0.31514,-1.2243,-0.534246,-1.44341
8,frc2342,0.0,2.95087,3.46602,0.0,0.0,0.0,-0.189451,0.201802,2.80252,0.202153,5.11499,-0.100729,2.95087,-0.162454,11.3835,14.5362,2.31733,1.83836,0.0553357,4.21103
9,frc246,0.0,-0.348955,4.5299,0.0,0.0,0.0,0.349632,-0.78531,3.423,0.151889,3.75951,-0.776037,-0.348955,0.392619,11.7124,10.5781,1.11103,-0.42475,0.710701,1.39698
